In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from numpy import sqrt

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

## Load data

In [ ]:
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hệ khuyến nghị/kem_review.csv')
product = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hệ khuyến nghị/kem_info.csv')

In [ ]:
product[["product_id", "name", "description", "price", "avg_rating", "luot_mua"]].head(3)

,product_id,name,description,price,avg_rating,luot_mua
0,19762237765,"Kem Chống Nắng Cho Da Dầu, Mụn, Nám Và Tăng Sắ...",Null,169.000 - ₫399.000,5.0,2kđã bán
1,17061960863,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...,Null,419.000,4.9,"10,9kđã bán"
2,5386949803,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...,"""Vaseline tin rằng bất cứ ai cũng xứng đáng sở...",158.000,4.9,"101,5kđã bán"


## Hàm lấy dữ liệu

In [ ]:
def getRating(user, productid):
    return ratings.loc[(ratings.username == user) & (ratings.product_id == productid), 'rating'].iloc[0]
    
def getProductId(user):
    return (ratings.loc[(ratings.username==user),'product_id'].tolist())

def getProductTitle(product_id):
    return (ratings.loc[(ratings.product_id == product_id),'product_name'].iloc[0])

def getusername(product_id):
    return (ratings.loc[(ratings.product_id==product_id),'username'].tolist())

def get_average_rating_user(user):
    rating_user = ratings.loc[((ratings.username== user)), 'rating']
    return np.mean(rating_user)

def get_average_rating_product(productid):
    rating_product = ratings.loc[((ratings.product_id== productid)), 'rating']
    return np.mean(rating_product)

In [ ]:
df = pd.read_csv('/content/username (3).csv')


In [ ]:
ratings = pd.merge(df, ratings, on='username', how='inner')
ratings= ratings[["username", "product_id", "rating", "product_name"]]
ratings

,username,product_id,rating,product_name
0,t*****1,1.706196e+10,5.0,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...
1,t*****1,1.706196e+10,4.0,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...
2,t*****1,1.706196e+10,5.0,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...
3,t*****1,1.706196e+10,5.0,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...
4,t*****1,1.706196e+10,5.0,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...
...,...,...,...,...
34308,t*****v,4.278990e+09,5.0,Kem chống nắng Năng động Cancer Council Active...
34309,t*****v,5.057939e+09,5.0,Kem chống nắng cho da nhạy cảm Altruist Dermat...
34310,t*****v,1.867873e+09,5.0,Kem chống nắng La roche posay fluid Shaka UV M...
34311,t*****v,3.380823e+09,5.0,Kem chống nắng dưỡng da PERFECT DIARY SPF50+ b...


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(ratings, test_size=0.003, random_state=42)

In [ ]:
X_test.shape

(103, 4)

## User-base

#### Cosine

In [ ]:
def cosine_similarity_user(user1, user2):
    both_watch_count = 0
    for productid in ratings.loc[ratings.username == user1, 'product_id'].to_list():
      for productid in ratings.loc[ratings.username == user2, 'product_id'].tolist():
        both_watch_count +=1
    if both_watch_count == 0:
      return 0

    rating1 = []
    rating2 = []

    for productid in ratings.loc[ratings.username == user1, 'product_id'].to_list():
      if productid in ratings.loc[ratings.username == user2, 'product_id'].tolist():
        rating1.append(getRating(user1, productid))
        rating2.append(getRating(user2, productid))

    if (norm(rating1)*norm(rating2)) == 0:
      return 0
    return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

#### Pearson

In [ ]:
def pearson_correlation_user(user1, user2):
    both_watch_count = []
    for productid in ratings.loc[(ratings.username == user1), 'product_id'].tolist():
        if productid in ratings.loc[(ratings.username == user2), 'product_id'].tolist():
            both_watch_count.append(productid)
    if len(both_watch_count) == 0:
        return 0
    rating_sum_1 = sum([getRating(user1, productid) for productid in both_watch_count])
    avg_rating_sum_1 = rating_sum_1/len(both_watch_count)

    rating_sum_2 = sum([getRating(user2, productid) for productid in both_watch_count])
    avg_rating_sum_2 = rating_sum_2/len(both_watch_count)

    #Gia tri tren tu
    numerator = sum([(getRating(user1, productid)- avg_rating_sum_1)*(getRating(user2, productid) - avg_rating_sum_2) for productid in both_watch_count])
    denominator = sqrt(sum([pow((getRating(user1, productid) - avg_rating_sum_1), 2) for productid in both_watch_count]))*sqrt(sum([pow((getRating(user2, productid) - avg_rating_sum_2), 2) for productid in both_watch_count]))
    if denominator == 0:
        return 0
    return numerator/denominator

### Top k user tương tự

In [ ]:
def get_similarity_user(user1, number_of_user, metric_similar):
    usernames = ratings.username.unique().tolist()

    similarity_score = []
    for user in usernames[50:100]:
        if (user != user1):
            if (metric_similar == "cosine"):
                similarity_score.append((cosine_similarity_user(user1, user), user))
            elif (metric_similar == "pearson"):
                similarity_score.append((pearson_correlation_user(user1, user), user))
    similarity_score.sort()
    similarity_score.reverse()
    return similarity_score[:number_of_user]

### Recommendation

#### Trung bình đánh giá

In [ ]:
def get_recommendation_tbdg_user(user1):
    usernames = ratings.username.unique().tolist()
    total = {}
    number_rating = {}
    similarity_sum = {}
    List_similar = get_similarity_user(user1, 20, "cosine")
    for peason, user in List_similar:
        for productid in getProductId(user):
            if productid not in getProductId(user1) or getRating(user1, productid) == 0:
                if productid not in total:
                    total[productid] = 0
                    number_rating[productid] = 0
                    total[productid] += getRating(user, productid)
                    if getRating(user, productid) != 0:
                        number_rating[productid] +=1
                else:
                    total[productid] += getRating(user, productid)
                    if getRating(user, productid) != 0:
                        number_rating[productid] +=1
   
    ranking = [(to/number_rating[productid], productid) for productid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(productid, score) for score, productid in ranking]

    return recomendations

#### Trung bình đánh giá có trọng số

In [ ]:
def get_recommendation_tbdgcts_user(user1):
    usernames = ratings.username.unique().tolist()
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_user(user1, 20, "cosine")
    for peason, user in List_similar:
        score = peason
        for productid in getProductId(user):
            if productid not in getProductId(user1) or getRating(user1, productid) == 0:
                if productid not in total:
                    total[productid] = 0
                    total[productid] += getRating(user, productid)*score
                    similarity_sum [productid] = 0
                    similarity_sum [productid] +=score
                else:
                    total[productid] += getRating(user, productid)*score
                    similarity_sum [productid] +=score
                
    ranking = [(to/similarity_sum[productid], productid) for productid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(productid, score) for score, productid in ranking]

    return recomendations

#### Khoảng đánh giá

In [ ]:
def get_recommendation_kdg_user(user1):
    username = ratings.username.unique().tolist()
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_user(user1, 20, "cosine")
    for peason, user in List_similar:
        score = peason
        for productid in getProductId(user):
            if productid not in getProductId(user1) or getRating(user1, productid) == 0:
                if productid not in total:
                    total[productid] = 0
                    total[productid] += (getRating(user, productid) - get_average_rating_user(user))*score
                    similarity_sum [productid] = 0
                    similarity_sum [productid] +=score
                else:
                    total[productid] += (getRating(user, productid) - get_average_rating_user(user))*score
                    similarity_sum [productid] +=score
                
    ranking = [((get_average_rating_user(user1) + to/similarity_sum[productid]), productid) for productid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(productid, score) for score, productid in ranking]

    return recomendations

#### Khuyến nghị

In [ ]:
from collections import defaultdict

def getRecommendationUser(user1):
    total = defaultdict(lambda:0)
    similaritySum = {}

    listUserPopular = get_similarity_user(user1, 10, "cosine")

    for score, user in listUserPopular:
        for productid in getProductId(user):
            # if movieId not in getMovieId(userId):
            rating = getRating(user, productid) * score
            if (total[productid] != 0 and total[productid] > rating):
                continue
            else:
                total[productid] = rating
                similaritySum[productid] = score

    ranking = [(round(tot/similaritySum[productid],1), productid) for productid,tot in total.items()]
    ranking.sort()
    ranking.reverse()
    
    recommendation = [(getProductTitle(productid), productid, score) for score, productid in ranking]
    return recommendation

In [ ]:
def get_recommendation_4test(test):

  X_test_pred = []
  X_test_username = X_test['username'].tolist()   #lấy ra danh sách các userID trong tập test 
  X_test_productid = X_test['product_id'].tolist()

  print("------Dự đoán cho tập test với {} điểm dữ liệu -----------".format(len(X_test_username)))
  
  for i in range(len(X_test_username)):        #lấy ra từng user
    list_R = getRecommendationUser(X_test_username[i])    #lất ra danh sách khuyến nghị cho mỗi USER 
    check = 0
    for j in list_R:
      if(X_test_productid[i] == j[0]):     #j[1] là lấy vị trí số 2 của 1 tuple 
        X_test_pred.append(j[1])
        check = 1
    if(check == 0):
      X_test_pred.append(0)
    print('----- Đang dự đoán cho dòng thứ {}'.format(i), 'là {}'.format(X_test_pred[i])) 
  return X_test_pred

In [ ]:
import time
start = time.time()

X_test_pred_rating = get_recommendation_4test(X_test)

end = time.time()
print('Tổng thời gian dự đoán: ', end - start)

In [ ]:
getRecommendationUser('h*****1')

[('Kem chống nắng vật lý lai hoá học cho mọi oại da Smile Leader Sun Cream 60ml Hàn Quốc Gia dụng Hồng Quế',
  21926989899.0,
  5.0),
 ('Bộ đôi kem chống nắng bảo vệ da khỏi tia UVA dài và UVB Anthelios UV Mune 400 50ml',
  21709729667.0,
  5.0),
 ('Serum Vaseline Gluta-Hya Dưỡng Da Sáng Mịn Sau 7 Ngày 330 ML ',
  20954628618.0,
  5.0),
 ('Kem chống nắng nâng tone ma:nyo Foundation-Free Sun Cream 15ml',
  19824983620.0,
  5.0),
 ('Kem Chống Nắng YOCOCI Chống Tia UVA/UVB, Dưỡng Trắng, Nâng Tone, Chống Trôi  Perfect Sun Cream SPF50+ PA++++ 50g YAHERA',
  19011591730.0,
  5.0),
 ('Kem Chống Nắng Collagen X3 Đông Anh 50gr [Hàng Công Ty] Chiết Xuất Nọc Ong, Chống Tia UV, Kiềm Dầu, Bảo Vệ Da Mịn Màng',
  17740977661.0,
  5.0),
 ('Sữa Chống Nắng Sunplay Skin Aqua UV SPF50+ PA+++', 17225176438.0, 5.0),
 ('[MUA 1 TẶNG 1 MASSK]Kem chống nắng vật lý lai hoá học cho da nhạy cảm Smile Leader Sun Cream 60ml Hàn Quốc',
  17121419120.0,
  5.0),
 ('Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA dài Anthe

### Dự đoán và đánh giá

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(X_test['rating'].tolist(), X_test_pred_rating)


0.8932038834951457

In [ ]:
import math

rmse = math.sqrt(mean_squared_error(X_test['rating'].tolist(), X_test_pred_rating))
print('----- RMSE: ',rmse)

----- RMSE:  0.9450946426126569
